# Algorithm 1

![alg1](Algorithm1_GenerativeModelForImage.png)

In [6]:
import pyprob #https://github.com/probprog/pyprob #!pip install pyprob
from pyprob import Model 
from pyprob.distributions import Categorical, Uniform

In [ ]:
class ImagesGen(Model):
  def __init__(self, name='ImagesModel', opt = None):
    super().__init__(name=name)
    self.opt = opt
    
    self.height, self.width = 128, 128
    self.colors = 5
    
  def forward(self):
    # Initialize images with 255 (white background)
    rendered_img = np.ones((self.height,self.width), dtype=np.uint8)*255
    
    num_objects = int(pyprob.sample(self.num_objects_distrs[label_idx], name = f"{class_label}_num_objects").item())

    # skip the "background" label in the z-order, since we intialize with 1s
    for class_label in self.z_order[1:]:
      id = self.label_ids[class_label]
      label_idx = id-2   #takes on values (1,11)

      # Need to record this value so we can loop through it down below
      num_objects = int(pyprob.sample(self.num_objects_distrs[label_idx], name = f"{class_label}_num_objects").item())

      for _ in range(num_objects):
        start_x = pyprob.sample(self.start_x_distrs[label_idx], name = f"{class_label}_{_}_start_x")
        start_y = pyprob.sample(self.start_y_distrs[label_idx], name = f"{class_label}_{_}_start_y")
        extent_x = pyprob.sample(self.extent_x_distrs[label_idx], name = f"{class_label}_{_}_extent_x") 
        extent_y = pyprob.sample(self.extent_y_distrs[label_idx], name = f"{class_label}_{_}_extent_y")
        
    
        pyprob.tag(start_x, name = f"{class_label}_{_}_start_x")
        pyprob.tag(start_y,name = f"{class_label}_{_}_start_y")
        pyprob.tag(extent_x,name = f"{class_label}_{_}_extent_x")
        pyprob.tag(extent_y,name = f"{class_label}_{_}_extent_y")

        rendered_img = self.draw_rectangle(rendered_img,(int(start_x), int(start_y)), (int(extent_x), int(extent_y)), class_label)

    rendered_img = torch.tensor(rendered_img) # I had to add this for some reason --> check if it is right
    pyprob.tag(rendered_img, name="semantic_image")
    one_hot_rendered_img = self.from_grey_to_one_hot(rendered_img)
    rendered_img = self.smooth_semantic(one_hot_rendered_img, alpha = 0.1)
    reshaped_rendered_img = rendered_img.view(12,-1).permute(1,0)
    pyprob.observe(Categorical(probs=reshaped_rendered_img),name='observed_semantic_image')


  def draw_rectangle(self, facade, start, extent, class_label):
    label = self.label_ids[class_label]
    rr, cc = rectangle(start, extent=extent, shape=facade.shape)
    facade[rr, cc] = label
    return facade


In [ ]:
import cv2
import os
import numpy as np
import random

img_size = 75
size = 5
question_size = 18  ## 2 x (6 for one-hot vector of color), 3 for question type, 3 for question subtype
q_type_idx = 12
sub_q_type_idx = 15
"""Answer : [yes, no, rectangle, circle, r, g, b, o, k, y]"""

nb_questions = 10
dirs = './data'

colors = [
    (0,0,255),##r
    (0,255,0),##g
    (255,0,0),##b
    (0,156,255),##o
    (128,128,128),##k
    (0,255,255)##y
]


try:
    os.makedirs(dirs)
except:
    print('directory {} already exists'.format(dirs))

def center_generate(objects):
    while True:
        pas = True
        center = np.random.randint(0+size, img_size - size, 2)        
        if len(objects) > 0:
            for name,c,shape in objects:
                if ((center - c) ** 2).sum() < ((size * 2) ** 2):
                    pas = False
        if pas:
            return center



def build_dataset():
    objects = []
    img = np.ones((img_size,img_size,3)) * 255
    for color_id,color in enumerate(colors):  
        center = center_generate(objects)
        if random.random()<0.5:
            start = (center[0]-size, center[1]-size)
            end = (center[0]+size, center[1]+size)
            cv2.rectangle(img, start, end, color, -1)
            objects.append((color_id,center,'r'))
        else:
            center_ = (center[0], center[1])
            cv2.circle(img, center_, size, color, -1)
            objects.append((color_id,center,'c'))